In [1]:
from analysis import get_market_cotIndex, get_market_participants
from utils import get_market_df, plot_df_chart, TRADERS_IN_FINANCIAL_FUTURES_FUT, LEGUACY_FUT, DISAGGREGATED_FUT, print_json

In [2]:
MARKET = "SILVER"
REPORT_TYPE = DISAGGREGATED_FUT

In [3]:
df = get_market_df(cot_report_type=REPORT_TYPE, market="SILVER")

loading data for the report type: disaggregated_fut ...
Selected: Disaggregated Report Futures-only - loading historical data ...
Selected: Disaggregated Report Futures-only from year: 2017 - loading data ...
Selected: Disaggregated Report Futures-only from year: 2018 - loading data ...
Selected: Disaggregated Report Futures-only from year: 2019 - loading data ...
Selected: Disaggregated Report Futures-only from year: 2020 - loading data ...
Selected: Disaggregated Report Futures-only from year: 2021 - loading data ...
Selected: Disaggregated Report Futures-only from year: 2022 - loading data ...
Selected: Disaggregated Report Futures-only from year: 2023 - loading data ...
Selected: Disaggregated Report Futures-only from year: 2024 - loading data ...
Selected: Disaggregated Report Futures-only from year: 2025 - loading data ...
Selected: Disaggregated Report Futures-only from year: 2026 - loading data ...


In [4]:
df.tail(4)

,market_and_exchange_names,as_of_date_in_form_yymmdd,cftc_contract_market_code,cftc_market_code,cftc_region_code,cftc_commodity_code,open_interest_all,prod_merc_positions_long_all,prod_merc_positions_short_all,swap_positions_long_all,...,conc_net_le_4_tdr_long_other,conc_net_le_4_tdr_short_other,conc_net_le_8_tdr_long_other,conc_net_le_8_tdr_short_other,contract_units,cftc_contract_market_code_quotes,cftc_market_code_quotes,cftc_commodity_code_quotes,cftc_subgroup_code,futonly_or_combined
Date,,,,,,,,,,,,,,,,,,,,,
2026-01-27,SILVER - COMMODITY EXCHANGE INC.,260127,084691,CMX,1,84,156637,5895,27108,28500,...,0.0,0.0,0.0,0.0,"(CONTRACTS OF 5,000 TROY OUNCES)",084691,CMX,84,N20,FutOnly
2026-02-03,SILVER - COMMODITY EXCHANGE INC.,260203,084691,CMX,1,84,143180,4847,23114,21173,...,0.0,0.0,0.0,0.0,"(CONTRACTS OF 5,000 TROY OUNCES)",084691,CMX,84,N20,FutOnly
2026-02-10,SILVER - COMMODITY EXCHANGE INC.,260210,084691,CMX,1,84,133641,4554,21344,21652,...,0.0,0.0,0.0,0.0,"(CONTRACTS OF 5,000 TROY OUNCES)",084691,CMX,84,N20,FutOnly
2026-02-17,SILVER - COMMODITY EXCHANGE INC.,260217,084691,CMX,1,84,131496,4959,20842,20839,...,0.0,0.0,0.0,0.0,"(CONTRACTS OF 5,000 TROY OUNCES)",084691,CMX,84,N20,FutOnly


- **Pick participants**

In [5]:
participants = get_market_participants(df)

list(participants.keys())

['prod_merc', 'swap', 'm_money', 'other_rept', 'tot_rept', 'nonrept']

In [6]:
participants = {k: v for k, v in participants.items() if k in ["m_money", "prod_merc"]}

print_json(participants)

{
        "prod_merc": {
                "oi_all": "open_interest_all",
                "long": "prod_merc_positions_long_all",
                "short": "prod_merc_positions_short_all",
                "long_oi": "pct_of_oi_prod_merc_long_all",
                "short_oi": "pct_of_oi_prod_merc_short_all",
                "long_change": "change_in_prod_merc_long_all",
                "short_change": "change_in_prod_merc_short_all"
        },
        "m_money": {
                "oi_all": "open_interest_all",
                "long": "m_money_positions_long_all",
                "short": "m_money_positions_short_all",
                "spread": "m_money_positions_spread_all",
                "long_oi": "pct_of_oi_m_money_long_all",
                "short_oi": "pct_of_oi_m_money_short_all",
                "spread_oi": "pct_of_oi_m_money_spread_all",
                "long_change": "change_in_m_money_long_all",
                "short_change": "change_in_m_money_short_all",
                "sp

- **Compute COT-index**

The Commitments of Traders (COT) Index is calculated as:

$$
\text{COT Index} = \left( \frac{P_{\text{current}} - P_{\min}}{P_{\max} - P_{\min}} \right) \times 100
$$

Where:

$ P_{\text{current}} $ :    Current Net Position <br>
$ P_{\min} $ :              Lowest Net Position in the past  N  period <br>
$ P_{\max} $ :              Highest Net Position in the past  N  periods <br>
$ N $ :                     Number of periods used in the lookback window


| Strategy                         | Recommended COT Index Period (N)    | Notes                                                           |
| -------------------------------- | ------------------------------------| --------------------------------------------------------------- |
| **Short-Term Trading**           | **13 weeks** (3 months)             | More sensitive to recent changes, but can be noisy.             |
| **Swing Trading**                | **26 weeks** (6 months)             | Most popular. Balances short-term moves and sentiment extremes. |
| **Position Trading / Long-Term** | **52 weeks** (1 year)               | Captures major sentiment extremes and long-term reversals.      |
| **Position Trading / Long-Term** | **156 weeks** (3 years)             ||
| **Position Trading / Long-Term** | **261 weeks** (5 years)             ||

In [ ]:
N = 261

df_index = get_market_cotIndex(market_df=df, participants=participants, window=N)

df_index.tail()

,prod_merc_Index(261),m_money_Index(261)
Date,,
2026-01-20,92.522625,46.731150
2026-01-27,88.919935,43.704113
2026-02-03,96.204441,40.045264
2026-02-10,99.852511,39.487546
2026-02-17,100.000000,41.372203


# **Plots**

- **plot of participants COT-index**

In [9]:
fig = plot_df_chart(
    df = df_index,
    chart_title = f"COT-index({N}) of different market participants",
    draw=False,
    use_markers=False
)

# Add green shaded area (e.g. for high index range)
fig.add_shape(
    type='rect',
    xref='paper',  # entire x-axis
    yref='y',
    x0=0,
    x1=1,
    y0=80,
    y1=100,
    fillcolor='green',
    opacity=0.2,
    line_width=0,
    layer='below'
)

# Add red shaded area (e.g. for low index range)
fig.add_shape(
    type='rect',
    xref='paper',
    yref='y',
    x0=0,
    x1=1,
    y0=0,
    y1=20,
    fillcolor='red',
    opacity=0.2,
    line_width=0,
    layer='below'
)

fig.show()